In [3]:
import os
import csv
import time
import urllib3
from datetime import datetime
from trino.dbapi import connect
from trino.auth import BasicAuthentication  # Import for LDAP authentication

# Disable SSL verification warnings
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

# Trino Connection Details
TRINO_HOST = "192.168.80.155"
TRINO_PORT = "30071"
TRINO_USER = "ctzn.bank"
TRINO_PASSWORD = "ctzn.bank_123"
TRINO_CATALOG = "iceberg"
TRINO_SCHEMA = "silver_crmuser"
TRINO_HTTP_SCHEME = "https"  # Secure connection

# Output CSV filename
OUTPUT_FILE = "ctzn_1_years_data.csv"

# SQL Query
SQL_QUERY = """ 
WITH all_combinations AS (
    -- Ensure all CIF_IDs from dim_gam and all 4 scheme types exist, and filter for the last 5 years of account opening
    SELECT DISTINCT 
        g.cif_id, 
        g.schm_type  
    FROM 
        gold.dim_gam AS g
    WHERE 
        -- Fix the date parsing by handling the microseconds format properly
        CAST(DATE_PARSE(SUBSTRING(g.acct_opn_date, 1, 19), '%Y-%m-%dT%H:%i:%s') AS DATE) >= CURRENT_DATE - INTERVAL '1' YEAR
), 

medians AS (
    SELECT 
        approx_percentile(total_debit_tran_vol, 0.5) AS median_dr_tran_volume,
        approx_percentile(total_credit_tran_vol, 0.5) AS median_cr_tran_volume,
        approx_percentile(total_debit_tran_count, 0.5) AS median_total_debit_transaction,
        approx_percentile(total_credit_tran_count, 0.5) AS median_total_credit_transaction
    FROM gold.mv_fact_deposit_account_insights
),

debit_credit_summary AS (
    SELECT 
        ac.cif_id, 
        ac.schm_type,
        -- BALANCE: Total savings (credits - debits)
        SUM(COALESCE(a.total_credit_tran_vol, 0)) - SUM(COALESCE(a.total_debit_tran_vol, 0)) AS balance,
        -- BALANCE_FREQUENCY: How often the balance is updated
        COALESCE(COUNT(DISTINCT a.nepali_month) / 12, 0) AS balance_frequency,
        -- PURCHASES: Total debit transactions
        SUM(COALESCE(a.total_debit_tran_vol, 0)) AS purchases,
        -- ONEOFF_PURCHASES: Maximum purchase amount in one-go
        MAX(COALESCE(a.total_debit_tran_vol, 0)) AS oneoff_purchases,
        -- INSTALLMENTS_PURCHASES: Total purchases excluding the highest transaction
        SUM(COALESCE(a.total_debit_tran_vol, 0)) - MAX(COALESCE(a.total_debit_tran_vol, 0)) AS installments_purchases,
        -- CASH_ADVANCE: Sum of overdraft transactions
        SUM(CASE WHEN ac.schm_type = 'ODA' THEN COALESCE(a.total_credit_tran_vol, 0) ELSE 0 END) AS cash_advance,
        -- PURCHASES_FREQUENCY: Monthly average debit transactions
        COALESCE(SUM(a.total_debit_tran_count) / NULLIF(COUNT(DISTINCT a.nepali_month), 0), 0) AS purchases_frequency,
        -- ONEOFF_PURCHASES_FREQUENCY: Frequency of large one-time purchases
        COALESCE(COUNT(DISTINCT CASE WHEN a.total_debit_tran_vol > 100000 THEN a.cif_id END) / NULLIF(COUNT(DISTINCT a.nepali_month), 0), 0) AS oneoff_purchases_frequency,
        -- PURCHASES_INSTALLMENTS_FREQUENCY: Frequency of installment purchases
        COALESCE(
            (SUM(a.total_debit_tran_count) - MAX(a.total_debit_tran_count)) / NULLIF(SUM(a.total_debit_tran_count), 0),
            0
        ) AS purchases_installments_frequency,
        -- CASH_ADVANCE_FREQUENCY: Frequency of cash advances
        COALESCE(COUNT(DISTINCT CASE WHEN a.total_credit_tran_vol > 0 THEN a.nepali_month END) / NULLIF(COUNT(DISTINCT a.nepali_month), 0), 0) AS cash_advance_frequency,
        -- CASH_ADVANCE_TRX: Count of overdraft transactions
        COALESCE(COUNT(DISTINCT CASE WHEN ac.schm_type = 'ODA' THEN a.foracid END), 0) AS cash_advance_trx,
        -- PURCHASES_TRX: Count of purchases for SBA and TDA schemes
        COALESCE(COUNT(DISTINCT CASE WHEN ac.schm_type IN ('SBA', 'TDA') THEN a.foracid END), 0) AS purchases_trx,
        -- PAYMENTS: Total credit transactions (all incoming credits)
        COALESCE(SUM(a.total_credit_tran_vol), 0) AS payments
    FROM 
        all_combinations AS ac
    LEFT JOIN 
        gold.mv_fact_deposit_account_insights AS a 
    ON 
        ac.cif_id = a.cif_id  
    GROUP BY 
        ac.cif_id, ac.schm_type
),

customer_info AS (
    SELECT 
        g.cif_id,
        COALESCE(CAST(SUBSTR(c.cust_dob, 1, 4) AS INTEGER), 2000) AS birth_year,
        EXTRACT(YEAR FROM CURRENT_DATE) - COALESCE(CAST(SUBSTR(c.cust_dob, 1, 4) AS INTEGER), 2000) AS tenure,
        COALESCE(AVG(c.salary_per_month) * 3, 0) AS credit_limit -- Estimated credit limit
    FROM 
        gold.dim_gam g
    LEFT JOIN 
        gold.dim_customers c 
    ON 
        g.cif_id = c.cif_id
    GROUP BY 
        g.cif_id, c.cust_dob
),

full_payment_summary AS (
    SELECT 
        dcs.cif_id,
        COALESCE(
            COUNT(DISTINCT CASE WHEN dcs.payments > dcs.purchases THEN dcs.cif_id END) / NULLIF(COUNT(DISTINCT dcs.cif_id), 0),
            0
        ) AS prc_full_payment
    FROM debit_credit_summary dcs
    GROUP BY dcs.cif_id
)

SELECT 
    dcs.*, 
    ci.tenure,
    -- MINIMUM_PAYMENTS: 5% of credit limit
    COALESCE(ci.credit_limit * 0.05, 0) AS minimum_payments,
    -- PRC_FULL_PAYMENT: Now correctly aggregated before joining
    COALESCE(fps.prc_full_payment, 0) AS prc_full_payment,
    COALESCE(ci.credit_limit, 0) AS credit_limit
FROM 
    debit_credit_summary dcs
LEFT JOIN 
    customer_info ci 
ON 
    dcs.cif_id = ci.cif_id
LEFT JOIN 
    full_payment_summary fps 
ON 
    dcs.cif_id = fps.cif_id
ORDER BY 
    dcs.cif_id
"""

def fetch_data():
    """Fetches data from Trino and saves it locally as a CSV file."""
    
    try:
        print("Connecting to Trino with LDAP authentication...")
        conn = connect(
            host=TRINO_HOST,
            port=TRINO_PORT,
            user=TRINO_USER,
            auth=BasicAuthentication(TRINO_USER, TRINO_PASSWORD),
            catalog=TRINO_CATALOG,
            schema=TRINO_SCHEMA,
            http_scheme=TRINO_HTTP_SCHEME,  # Use HTTPS
            request_timeout=600,  # Increase timeout to 10 minutes
            verify=False  # <--- This disables SSL verification
        )
        cursor = conn.cursor()

        print("Executing query...")
        cursor.execute(SQL_QUERY)

        # Fetch column headers
        columns = [desc[0] for desc in cursor.description]

        # Save data to CSV file locally
        with open(OUTPUT_FILE, "w", newline="", encoding="utf-8") as f:
            writer = csv.writer(f)
            writer.writerow(columns)  # Write headers
            
            # Fetch rows in streaming mode
            while True:
                rows = cursor.fetchmany(1000)  # Fetch 1000 rows at a time
                if not rows:
                    break
                writer.writerows(rows)

        # Close connection
        conn.close()
        print(f"Data successfully saved to {OUTPUT_FILE}")
        return OUTPUT_FILE  

    except Exception as e:
        print(f"Error: {str(e)}")
        return None

# Run the function
csv_file_path = fetch_data()
print(f"CSV file is saved at: {csv_file_path}")


Connecting to Trino with LDAP authentication...
Executing query...
Data successfully saved to ctzn_1_years_data.csv
CSV file is saved at: ctzn_1_years_data.csv


In [2]:
pip install trino

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 84.9 kB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 177.7 kB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.
